In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [2]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [3]:
link = 'http://www.fapeg.go.gov.br/categoria/editais/'
parser_general = parser(link)
numpags = parser_general.xpath('//a[@class="page-numbers"]/text()')
numpags = int(numpags[-1])
numpags

titulos_proyectos = []
links_proyectos = []
fechas_proyectos = []
n = 1
while n <= numpags:

    link_pag = link + 'page/' + str(n) + '/'
    

    parser_pag = parser(link_pag)

    title = parser_pag.xpath('//h1[@class="entry-title"]/a/text()')

    link_proy = parser_pag.xpath('//h1[@class="entry-title"]/a/@href')

    date = parser_pag.xpath('//time[@class="entry-date"]/text()')

    titulos_proyectos += title
    links_proyectos += link_proy
    fechas_proyectos += date

    n+=1 

In [19]:
pdfs_proyectos = []

emitir = ['http://www.fapeg.go.gov.br/wp-content/uploads/2020/01/DECRETO-NUMERADO-Nº-9.597.pdf']

for link in links_proyectos:
    
    parser_proyecto = parser(link)

    pdfs_brutos= parser_proyecto.xpath('//a/@href')

    pdfs_proyecto = ''

    for pdf in pdfs_brutos:
        if pdf.endswith('.pdf') and pdf not in emitir:
            if pdf.startswith('https://') or pdf.startswith('http://'):
                pdfs_proyecto = pdfs_proyecto + pdf + ', '
            else:
                pdf = 'http://www.fapeg.go.gov.br' + pdf
                pdfs_proyecto = pdfs_proyecto + pdf + ', '
        
    pdfs_proyectos.append(pdfs_proyecto.strip(', '))

In [20]:
fapeg = pd.DataFrame()

fapeg['Título'] = titulos_proyectos
fapeg['Link'] = links_proyectos
fapeg['Fecha'] = fechas_proyectos
fapeg['Pdfs'] = pdfs_proyectos

fapeg.to_excel('brasil_Fapeg.xlsx')

In [21]:
fapeg[0:21]

,Título,Link,Fecha,Pdfs
0,Chamada Pública Nº03/2021 – Programa de Conces...,http://www.fapeg.go.gov.br/chamada-03-2021-pro...,14 de junho de 2021,http://www.fapeg.go.gov.br/wp-content/uploads/...
1,Chamada Pública Nº 02/2021- Seleção de Bolsist...,http://www.fapeg.go.gov.br/chamada-publica-no-...,6 de abril de 2021,http://www.fapeg.go.gov.br/wp-content/uploads/...
2,CHAMADA CONFAP CNPq FAPEG – ERC 2020,http://www.fapeg.go.gov.br/chamada-confap-cnpq...,2 de fevereiro de 2021,http://www.fapeg.go.gov.br/wp-content/uploads/...
3,EDITAL FAPEG Nº01/2021 PROGRAMA DE CONCESSÃO D...,http://www.fapeg.go.gov.br/edital-fapeg-no01-2...,5 de janeiro de 2021,http://www.fapeg.go.gov.br/wp-content/uploads/...
4,CONFAP – CNPq – H2020 Green Deal Call,http://www.fapeg.go.gov.br/confap-cnpq-h2020-g...,5 de novembro de 2020,http://www.fapeg.go.gov.br/wp-content/uploads/...
5,Chamada Nº 06/2020 – Seleção de Bolsistas no â...,http://www.fapeg.go.gov.br/chamada-no-06-2020-...,31 de agosto de 2020,http://www.fapeg.go.gov.br/wp-content/uploads/...
6,Chamada Pública 05/2020 – 7ª Edição do PROGRAM...,http://www.fapeg.go.gov.br/chamada-publica-05-...,26 de agosto de 2020,http://www.fapeg.go.gov.br/wp-content/uploads/...
7,Chamada Pública Nº 04/2020 – Seleção de profi...,http://www.fapeg.go.gov.br/chamada-publica-no-...,21 de agosto de 2020,http://www.fapeg.go.gov.br/wp-content/uploads/...
8,Chamada Pública 03/2020 – Programa Governo com...,http://www.fapeg.go.gov.br/chamada-publica-03-...,21 de julho de 2020,http://www.fapeg.go.gov.br/wp-content/uploads/...
9,Chamada Confap-MSCA Rise 2020,http://www.fapeg.go.gov.br/chamada-confap-msca...,26 de março de 2020,http://www.fapeg.go.gov.br/wp-content/uploads/...
